Получить максимальную метрику качества (R2, RMSE).

Введение
Целью данной задачи является прогнозирование суммы пятничных распродаж в магазинах сети Walmart с помощью построения регрессионных моделей и их анализа.
Набор данных состоит из исторических данных, которые охватывают продажи с 2010-02-05 по 2012-11-01, предствленных в файле Walmart.csv
#Использовать приплайны.
#Выполнить поиск лучших параметров
#Кроссвалидация.
#Сохранить и загрузить моделль. 

V2
15.01.24
Anatoly

In [3]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from hyperopt import fmin, tpe, rand, hp, STATUS_OK, Trials
file_path_test ="C:/Users/qiarr/Desktop/task/lesson 20/Walmart.csv"
data = pd.read_csv(file_path_test, encoding='cp1251', )



In [16]:
data.head()

,Store,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year
0,1.0,0.0,42.31,2.572,211.096358,8.106,0.422146
1,1.0,1.0,38.51,2.548,211.242170,8.106,1.005479
2,1.0,0.0,39.93,2.514,211.289143,8.106,0.218721
3,1.0,0.0,46.63,2.561,211.319643,8.106,0.237900
4,1.0,0.0,46.50,2.625,211.350143,8.106,0.424886


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


In [ ]:
"""X = df_test.drop("Weekly_Sales", axis=1) 
y = df_test[["Weekly_Sales"]]
X = X.drop(["Date", "Store"], axis=1)

logreg = LinearRegression()

scores = cross_val_score(logreg, X, y, cv=5)
print("Значения правильности перекрестной проверки: {}".format(scores))"""

'X = df_test.drop("Weekly_Sales", axis=1)  # Удаляем столбец "Unemployment" из набора данных X\ny = df_test[["Weekly_Sales"]]\nX = X.drop(["Date", "Store"], axis=1)\n\nlogreg = LinearRegression()\n\nscores = cross_val_score(logreg, X, y, cv=5)\nprint("Значения правильности перекрестной проверки: {}".format(scores))'

In [13]:
data['Date'] = pd.to_datetime(data['Date'], format='mixed')

data['Month'] = data['Date'].dt.month.astype(dtype=np.float64)
data['Day'] = data['Date'].dt.day.astype(dtype=np.float64)
data['Year'] = data['Date'].dt.year.astype(dtype=np.float64)
data.drop('Date', axis=1, inplace=True)
data = data.astype(dtype=np.float64)
data['Year'] = data['Day']/365 + data['Month']/12
data.drop(['Month', 'Day'], axis=1, inplace=True)
columns_outliers = ['Temperature', 'Unemployment']
y = data.pop('Weekly_Sales')
x = data

In [15]:
x.head()

,Store,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year
0,1.0,0.0,42.31,2.572,211.096358,8.106,0.422146
1,1.0,1.0,38.51,2.548,211.242170,8.106,1.005479
2,1.0,0.0,39.93,2.514,211.289143,8.106,0.218721
3,1.0,0.0,46.63,2.561,211.319643,8.106,0.237900
4,1.0,0.0,46.50,2.625,211.350143,8.106,0.424886


In [14]:
y.head()

0    1643690.90
1    1641957.44
2    1611968.17
3    1409727.59
4    1554806.68
Name: Weekly_Sales, dtype: float64

In [ ]:
# Создание объекта модели
model = LinearRegression()

# Задание сетки параметров для поиска
param_grid = {
    'fit_intercept': [True, False],
    
}

# Создание объекта GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)

# Обучение модели с использованием поиска по сетке
grid_search.fit(X, y)


# Извлечение лучшей модели
best_model = grid_search.best_estimator_


cv_scores = cross_val_score(best_model, X, y, cv=5)

# Оценка производительности модели на тестовом наборе данных
accuracy = grid_search.best_estimator_.score(X, y)
print("Оценки кросс-валидации с лучшей моделью:", cv_scores)
print("Средняя оценка кросс-валидации:", cv_scores.mean())

Оценки кросс-валидации с лучшей моделью: [ 0.0030215  -0.27409073 -0.31392674 -0.49163956 -1.05034942]
Средняя оценка кросс-валидации: -0.4253969883484034


In [ ]:
# Создание объекта модели
model = RandomForestRegressor(n_estimators=100, random_state=42)

cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
# Преобразование отрицательных значений MSE в положительные значения
mse_scores = -cv_scores

# Вывод оценок кросс-валидации и их среднего значения
print("Оценки кросс-валидации с лучшей моделью:", mse_scores)
print("Среднее значение кросс-валидации:", mse_scores.mean())


Оценки кросс-валидации с лучшей моделью: [8.67355083e+11 1.00375903e+12 4.28671852e+11 3.19279304e+11
 9.01536485e+11]
Среднее значение кросс-валидации: 704120350499.2673


In [ ]:
# Создание модели градиентного бустинга
gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Оценка производительности модели с помощью кросс-валидации
cv_scores = cross_val_score(gb_regressor, X, y, cv=5, scoring='neg_mean_squared_error')

# Преобразование отрицательных значений MSE в положительные значения
mse_scores = -cv_scores

# Вывод оценок кросс-валидации и их среднего значения
print("Оценки кросс-валидации (MSE):", mse_scores)
print("Среднее значение MSE:", mse_scores.mean())

Оценки кросс-валидации (MSE): [5.54144435e+11 7.32533414e+11 3.50436173e+11 3.31362742e+11
 5.56131016e+11]
Среднее значение MSE: 504921556121.7004


In [ ]:
# Linear Regression
linear_regression = LinearRegression()
print("Linear Regression hyperparameters:")
print(linear_regression.get_params())

# Ridge Regression
ridge_regression = Ridge()
print("Ridge Regression hyperparameters:")
print(ridge_regression.get_params())

# Lasso Regression
lasso_regression = Lasso()
print("Lasso Regression hyperparameters:")
print(lasso_regression.get_params())

# Random Forest Regression
random_forest = RandomForestRegressor()
print("Random Forest Regression hyperparameters:")
print(random_forest.get_params())

# Gradient Boosting Regression
gradient_boosting = GradientBoostingRegressor()
print("Gradient Boosting Regression hyperparameters:")
print(gradient_boosting.get_params())

# Support Vector Regression (SVR)
svr = SVR()
print("SVR hyperparameters:")
print(svr.get_params())

Linear Regression hyperparameters:
{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}
Ridge Regression hyperparameters:
{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'positive': False, 'random_state': None, 'solver': 'auto', 'tol': 0.0001}
Lasso Regression hyperparameters:
{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'max_iter': 1000, 'positive': False, 'precompute': False, 'random_state': None, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}
Random Forest Regression hyperparameters:
{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
Gradient Boosting Regression hyperparameters:
{'alpha': 0.9, 'cc

спустя большое количетсво попытко с очень страными метриками я подумал что лучше всего попробовать найти и модель и ее параметры

In [ ]:
from sklearn.metrics import make_scorer

# Определение функции для оптимизации
def optimize(params):
    model = None
    if params['model']['model'] == 'LinearRegression':
        model = LinearRegression(**params['model']['params'])
    elif params['model']['model'] == 'Ridge':
        model = Ridge(**params['model']['params'])
    elif params['model']['model'] == 'Lasso':
        model = Lasso(**params['model']['params'])
    elif params['model']['model'] == 'RandomForest':
        model = RandomForestRegressor(**params['model']['params'], random_state=42)
    elif params['model']['model'] == 'GradientBoosting':
        model = GradientBoostingRegressor(**params['model']['params'], random_state=42)
    elif params['model']['model'] == 'SVR':
        model = SVR(**params['model']['params'])
    
    
    def rmse(y_true, y_pred):
        return np.sqrt(mean_squared_error(y_true, y_pred))
    rmse_scorer = make_scorer(rmse, greater_is_better=False)   
    # Выполнение кросс-валидации
    mse_scores = cross_val_score(model, x, y, cv=5, scoring=rmse_scorer)
    
    



    avg_mse = np.mean(mse_scores)
    
    return {'loss': avg_mse, 'status': STATUS_OK}

# Определение пространства поиска гиперпараметров и моделей
param_space = {
    'model': hp.choice('model', [
        {
            'model': 'LinearRegression',
            'params': {
                'fit_intercept': hp.choice('lr_fit_intercept', [True, False]),
                
            }
        },
        {
            'model': 'Ridge',
            'params': {
                'alpha': hp.uniform('ridge_alpha', 0.1, 2.0)
            }
        },
        {
            'model': 'Lasso',
            'params': {
                'alpha': hp.uniform('lasso_alpha', 0.1, 2.0)
            }
        },
        {
            'model': 'RandomForest',
            'params': {
                'n_estimators': hp.choice('rf_n_estimators', range(100, 1000)),
                'max_depth': hp.choice('rf_max_depth', [5, 10, 20]),
                'min_samples_split': hp.choice('rf_min_samples_split', [2, 5, 10])
            }
        },
        {
            'model': 'GradientBoosting',
            'params': {
                'n_estimators': hp.choice('gb_n_estimators', range(100, 1000)),
                'learning_rate': hp.uniform('gb_learning_rate', 0.01, 0.2),
                'max_depth': hp.choice('gb_max_depth', [3, 5, 7]),
                'subsample': hp.uniform('gb_subsample', 0.5, 1)
            }
        },
        {
            'model': 'SVR',
            'params': {
                'C': hp.uniform('svr_C', 0.1, 2.0),
                'kernel': hp.choice('svr_kernel', ['linear', 'rbf'])
            }
        }
    ])
}

# Запуск процесса оптимизации
trials = Trials()
best_params = fmin(fn=optimize, space=param_space, algo=tpe.suggest, max_evals=100, trials=trials)

print("Лучшие гиперпараметры и модель:", best_params)

100%|██████████| 100/100 [44:17<00:00, 26.57s/trial, best loss: -695025.1461997401]  
Лучшие гиперпараметры и модель: {'gb_learning_rate': 0.1976105559309828, 'gb_max_depth': 0, 'gb_n_estimators': 311, 'gb_subsample': 0.8322871730072593, 'model': 4}


Лучшие гиперпараметры и модель: {'model': 5, 'svr_C': 1.9970350315307233, 'svr_kernel': 1} при метрике nmse

Лучшие гиперпараметры и модель: {'gb_learning_rate': 0.1649446580353066, 'gb_max_depth': 2, 'gb_n_estimators': 839, 'gb_subsample': 0.5040348698621966, 'model': 4} при метрике rmse спустя 31 минуту поиска

Лучшие гиперпараметры и модель: {'gb_learning_rate': 0.1786689023119613, 'gb_max_depth': 2, 'gb_n_estimators': 608, 'gb_subsample': 0.5013566097870894, 'model': 4} при обновленном датасете за 34 минуты


Лучшие гиперпараметры и модель: {'gb_learning_rate': 0.1976105559309828, 'gb_max_depth': 0, 'gb_n_estimators': 311, 'gb_subsample': 0.8322871730072593, 'model': 4} моя 4ая попытка на 40 минут поисков где я перед этим подправил данные

In [ ]:

# Создание модели Support Vector Regression (SVR) с указанными гиперпараметрами
svr_regressor = SVR(kernel='rbf', C=1.9970350315307233)  # Используем ядро 'rbf' и параметр C
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
# Оценка производительности модели с помощью кросс-валидации

rmse_scorer = make_scorer(rmse, greater_is_better=False)
# Преобразование отрицательных значений MSE в положительные значения
cv_scores = cross_val_score(svr_regressor, x, y, cv=5, scoring=rmse_scorer)

# Вывод результатов кросс-валидации
print("Оценки кросс-валидации (RMSE):", cv_scores)
print("Среднее значение RMSE:", np.mean(cv_scores))

Оценки кросс-валидации (RMSE): [-682376.55236947 -684164.67531221 -618949.83808112 -455317.43709286
 -480874.88216871]
Среднее значение RMSE: -584336.6770048735


In [ ]:
# Лучшие гиперпараметры для градиентного бустинга
best_gb_params = {
    'n_estimators': 839,
    'learning_rate': 0.1649446580353066,
    'max_depth': 2,
    'subsample': 0.5040348698621966
}

# Создание модели градиентного бустинга с указанными гиперпараметрами
gb_regressor = GradientBoostingRegressor(n_estimators=best_gb_params['n_estimators'],
                                         learning_rate=best_gb_params['learning_rate'],
                                         max_depth=best_gb_params['max_depth'],
                                         subsample=best_gb_params['subsample'],
                                         random_state=42)

# Оценка производительности модели с помощью кросс-валидации
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

# Преобразование отрицательных значений MSE в положительные значения
cv_scores = cross_val_score(gb_regressor, x, y, cv=5, scoring=rmse_scorer)

# Вывод результатов кросс-валидации
print("Оценки кросс-валидации (RMSE):", cv_scores)
print("Среднее значение RMSE:", np.mean(cv_scores))

Оценки кросс-валидации (RMSE): [-1026810.59160529  -836839.38502733  -524968.00460675  -560764.90542522
  -513392.75914585]
Среднее значение RMSE: -692555.1291620876


In [ ]:
# Лучшие гиперпараметры для градиентного бустинга
best_gb_params = {
    'n_estimators': 311,
    'learning_rate': 0.1976105559309828,
    'max_depth': 1,
    'subsample': 0.8322871730072593
}

# Создание модели градиентного бустинга с указанными гиперпараметрами
gb_regressor = GradientBoostingRegressor(n_estimators=best_gb_params['n_estimators'],
                                         learning_rate=best_gb_params['learning_rate'],
                                         max_depth=best_gb_params['max_depth'],
                                         subsample=best_gb_params['subsample'],
                                         random_state=42)

# Оценка производительности модели с помощью кросс-валидации
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

# Преобразование отрицательных значений MSE в положительные значения
cv_scores = cross_val_score(gb_regressor, x, y, cv=5, scoring=rmse_scorer)

# Вывод результатов кросс-валидации
print("Оценки кросс-валидации (RMSE):", cv_scores)
print("Среднее значение RMSE:", np.mean(cv_scores))

Оценки кросс-валидации (RMSE): [-1009242.13713756  -788548.07418574  -596960.45330393  -538010.69070411
  -505439.03005895]
Среднее значение RMSE: -687640.0770780571


Оценки кросс-валидации (RMSE): [-1009242.13713756  -788548.07418574  -596960.45330393  -538010.69070411
  -505439.03005895]
Среднее значение RMSE: -687640.0770780571 это лучшее значение что я смог получить,не знаю почему но даже подбром лушмх моделей и их параметров получилось так

В итоге этого задания я изучил самые разные метрики оенки даже те о которых не слышал а также сам в удивление для себя вроде бы смог построить алгоритм по подбору моделе и их параметров вначале основывая на одной метрике NMSE ,после основываясь на метрике RMSE

In [ ]:
import pickle

# Сохранение модели в файл с помощью pickle
with open('gb_regressor_model.pkl', 'wb') as f:
    pickle.dump(gb_regressor, f)

я сохранил модель gb_regressor_model